In [13]:
import numpy as np
import json
import pandas as pd
import pickle
import xgboost as xgb

In [14]:
with open('../data/output_data/model.pickle', 'rb') as f:
    model = pickle.load(f)
with open('../data/output_data/candis_list.pickle', 'rb') as f:
    candis_list = pickle.load(f)
with open('../data/output_data/label_encoder.pickle', 'rb') as f:
    le = pickle.load(f)
with open('../data/input_data/test_q.json', encoding='utf-8') as f:
    test_list = json.load(f)

test_X = pd.read_pickle('../data/output_data/test_df.pickle')
test_X = test_X.reindex(test_X.columns.astype('object'), axis=1)
dtest = xgb.DMatrix(test_X)

In [15]:
candis_list

[[400, 175, 534, 403, 16, 187, 318, 444, 612, 533],
 [36, 186, 662, 229, 827, 83, 164, 46, 610, 121, 264],
 [641, 762, 799, 622, 522, 624, 575, 792, 832, 273, 499],
 [532, 134, 336, 707, 558, 138, 323, 825, 61, 417, 295],
 [322, 549, 522, 156, 151, 301, 714, 396, 282, 499, 140],
 [279, 186, 707, 152, 229, 264, 81, 164, 577, 848, 303],
 [170, 127, 686, 102, 462, 302, 516, 842, 642, 508, 520],
 [700, 483, 719, 706, 258, 677, 850, 64, 261, 774, 788],
 [241, 267, 155, 346, 442, 517, 285, 421, 845, 410, 143],
 [386, 424, 628, 821, 430, 539, 446, 813, 434, 632, 511],
 [166, 496, 501, 747, 862, 19, 470, 486, 551, 728, 409],
 [404, 571, 90, 563, 299, 863, 44, 361, 253, 569, 347],
 [153, 69, 53, 832, 151, 79, 522, 107, 621, 714, 641],
 [522, 624, 79, 153, 523, 792, 119, 574, 301, 273, 379],
 [849, 596, 573, 539, 435, 13, 536, 440, 456, 59, 284],
 [749, 799, 356, 714, 396, 389, 622, 67, 32, 480, 589],
 [813, 175, 628, 386, 821, 573, 536, 533, 435, 424, 415],
 [659, 711, 859, 790, 202, 458, 804, 

In [16]:
pred = model.predict(dtest)
# np.argmax(pred, axis=1)
pred

array([[3.8197450e-05, 1.1080566e-04, 1.1083229e-04, ..., 1.6893839e-05,
        5.2552882e-06, 1.1083229e-04],
       [5.7700749e-06, 1.3271520e-05, 1.3274711e-05, ..., 4.2071292e-06,
        1.9269621e-11, 1.3274711e-05],
       [1.8588703e-07, 2.1619940e-07, 2.1625137e-07, ..., 8.9078384e-08,
        1.0355470e-07, 2.1625137e-07],
       ...,
       [2.8550226e-05, 1.4329168e-04, 1.4332611e-04, ..., 1.8172905e-04,
        8.2605718e-09, 1.4332611e-04],
       [1.4309461e-06, 1.1264158e-06, 1.1266866e-06, ..., 2.0047229e-07,
        1.0445502e-09, 1.1266866e-06],
       [3.9519961e-08, 5.1512885e-08, 5.1525266e-08, ..., 2.5368434e-08,
        6.5513809e-08, 5.1525266e-08]], dtype=float32)

In [17]:
ans_list = []
for i, data in enumerate(pred):
    score_dict = {}
    for candis in candis_list[i]:
        auth_id = le.inverse_transform([candis])[0]
        score_dict[auth_id] = data[candis]

    sorted_list = sorted(score_dict.items(), key=lambda x:x[1], reverse=True)
    candis_ans_list = [auth_id[0] for auth_id in sorted_list]
    ans_list.append({
        'title_id': test_list[i]['title_id'],
        'candidates': candis_ans_list
    })

print(f'test: {len(test_list)}')
print(f'ans: {len(ans_list)}')

test: 548
ans: 548


In [18]:
with open('../data/output_data/suggestion.json', 'w', encoding='utf-8') as f:
    json.dump(ans_list, f)